In [7]:
%pip install python-binance

Note: you may need to restart the kernel to use updated packages.


In [8]:
api_key =  '9c461c13ec4b4491b4d5905948bb7805313adca82287ca6997405972c026a0e5'
api_secret= 'cd8eb32222b99586ca8967eb7325b325f5ab0a4f4d2aa536bdd215aa01801ea5'

In [9]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
import time

In [10]:
client = Client(api_key = api_key, api_secret = api_secret, tld = "com", testnet = True)
client.futures_account()

{'feeTier': 0,
 'canTrade': True,
 'canDeposit': True,
 'canWithdraw': True,
 'updateTime': 0,
 'totalInitialMargin': '5.86110000',
 'totalMaintMargin': '0.58611000',
 'totalWalletBalance': '14999.36472102',
 'totalUnrealizedProfit': '0.42481818',
 'totalMarginBalance': '14999.78953920',
 'totalPositionInitialMargin': '5.86110000',
 'totalOpenOrderInitialMargin': '0.00000000',
 'totalCrossWalletBalance': '14999.36472102',
 'totalCrossUnPnl': '0.42481818',
 'availableBalance': '14993.66713920',
 'maxWithdrawAmount': '14993.66713920',
 'assets': [{'asset': 'BTC',
   'walletBalance': '0.00000000',
   'unrealizedProfit': '0.00000000',
   'marginBalance': '0.00000000',
   'maintMargin': '0.00000000',
   'initialMargin': '0.00000000',
   'positionInitialMargin': '0.00000000',
   'openOrderInitialMargin': '0.00000000',
   'maxWithdrawAmount': '0.00000000',
   'crossWalletBalance': '0.00000000',
   'crossUnPnl': '0.00000000',
   'availableBalance': '0.00000000',
   'marginAvailable': True,
   

In [11]:
class FuturesTrader():  # Triple SMA Crossover
    
    def __init__(self, symbol, bar_length, sma_s, sma_m, sma_l, units, position = 0, leverage = 5):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.leverage = leverage # NEW
        self.cum_profits = 0 # NEW
        #self.trades = 0 
        #self.trade_values = []
        
        #*****************add strategy-specific attributes here******************
        self.SMA_S = sma_s
        self.SMA_M = sma_m
        self.SMA_L = sma_l
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        client.futures_change_leverage(symbol = self.symbol, leverage = self.leverage) # NEW
        
        self.twm = ThreadedWebsocketManager(testnet = True) # testnet
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_futures_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length) # Adj: start_kline_futures_socket
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.futures_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000) # Adj: futures_historical_klines
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
        
        # print out
        print(".", end = "", flush = True) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        data = self.data.copy()
        
        #******************** define your strategy here ************************
        data = data[["Close"]].copy()
        
        data["SMA_S"] = data.Close.rolling(window = self.SMA_S).mean()
        data["SMA_M"] = data.Close.rolling(window = self.SMA_M).mean()
        data["SMA_L"] = data.Close.rolling(window = self.SMA_L).mean()
        
        data.dropna(inplace = True)
                
        cond1 = (data.SMA_S > data.SMA_M) & (data.SMA_M > data.SMA_L)
        cond2 = (data.SMA_S < data.SMA_M) & (data.SMA_M < data.SMA_L)
        
        data["position"] = 0
        data.loc[cond1, "position"] = 1
        data.loc[cond2, "position"] = -1
        #***********************************************************************
        
        self.prepared_data = data.copy()
    
    def execute_trades(self): # Adj! 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = 2 * self.units)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == -1:
                order = client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
        if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING SHORT") 
            elif self.position == 1:
                order = client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = 2 * self.units)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
    
    def report_trade(self, order, going): # Adj!
        
        time.sleep(0.1)
        order_time = order["updateTime"]
        trades = client.futures_account_trades(symbol = self.symbol, startTime = order_time)
        order_time = pd.to_datetime(order_time, unit = "ms")
        
        # extract data from trades object
        df = pd.DataFrame(trades)
        columns = ["qty", "quoteQty", "commission","realizedPnl"]
        for column in columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        base_units = round(df.qty.sum(), 5)
        quote_units = round(df.quoteQty.sum(), 5)
        commission = -round(df.commission.sum(), 5)
        real_profit = round(df.realizedPnl.sum(), 5)
        price = round(quote_units / base_units, 5)
        
        # calculate cumulative trading profits
        self.cum_profits += round((commission + real_profit), 5)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(order_time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(order_time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(order_time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")
        

In [12]:

symbol = "BTCUSDT"
bar_length = "1m"
sma_s = 10
sma_m = 20
sma_l = 50
units = 0.001
position = 0
leverage = 10

trader = FuturesTrader(symbol = symbol, bar_length = bar_length,
                       sma_s = sma_s, sma_m = sma_m, sma_l = sma_l, 
                       units = units, position = position, leverage = leverage)

trader.start_trading(historical_days = 1/24)



..............

----------------------------------------------------------------------------------------------------
2022-10-23 23:28:04.052000 | GOING SHORT
2022-10-23 23:28:04.052000 | Base_Units = 0.002 | Quote_Units = 38.522 | Price = 19261.0 
2022-10-23 23:28:04.052000 | Profit = -0.26879 | CumProfits = -0.2842 
----------------------------------------------------------------------------------------------------

.

----------------------------------------------------------------------------------------------------
2022-10-23 23:28:04.218000 | GOING SHORT
2022-10-23 23:28:04.218000 | Base_Units = 0.001 | Quote_Units = 19.261 | Price = 19261.0 
2022-10-23 23:28:04.218000 | Profit = -0.13439 | CumProfits = -0.14209 
----------------------------------------------------------------------------------------------------

.......................................

----------------------------------------------------------------------------------------------------
2022-10-23 23:31:04.373000 |

In [13]:
trader.twm.stop()
trader.prepared_data

,Close,SMA_S,SMA_M,SMA_L,position
Date,,,,,
2022-10-23 23:16:00,19449.9,19347.56,19357.280,19373.984,-1
2022-10-23 23:17:00,19448.0,19367.16,19357.180,19375.044,0
2022-10-23 23:18:00,19449.9,19372.15,19357.175,19376.142,0
2022-10-23 23:19:00,19449.9,19377.25,19364.670,19377.240,0
2022-10-23 23:20:00,19449.9,19399.74,19372.165,19378.338,0
2022-10-23 23:21:00,19449.9,19422.23,19383.650,19379.436,1
2022-10-23 23:22:00,19260.0,19408.23,19374.155,19376.736,0
2022-10-23 23:23:00,19250.0,19410.73,19364.160,19373.918,0
2022-10-23 23:24:00,19250.0,19390.74,19356.660,19371.946,0


.......